In [ ]:
# user input for positions/ location

In [ ]:
def get_user_input():
    # Get the date and time from the user
    year = int(input("Enter year (YYYY): "))
    month = int(input("Enter month (MM): "))
    day = int(input("Enter day (DD): "))
    hour = int(input("Enter hour (24h format): "))
    minute = int(input("Enter minute: "))
    
    # Get the geographical location
    latitude = float(input("Enter latitude (e.g., 40.7128 for NYC): "))
    longitude = float(input("Enter longitude (e.g., -74.0060 for NYC): "))
    
    return year, month, day, hour, minute, latitude, longitude

In [ ]:
# pulling planetary positions

In [ ]:
import swisseph as swe

def query_planetary_positions(year, month, day, hour, minute, latitude, longitude):
    # Convert date and time to Julian Day
    julian_day = swe.julday(year, month, day, hour + minute/60.0, swe.GREG_CAL)
    
    # List of planets to calculate
    planets = {
        "Sun": swe.SE_SUN,
        "Moon": swe.SE_MOON,
        "Mercury": swe.SE_MERCURY,
        "Venus": swe.SE_VENUS,
        "Mars": swe.SE_MARS,
        "Jupiter": swe.SE_JUPITER,
        "Saturn": swe.SE_SATURN,
        "Uranus": swe.SE_URANUS,
        "Neptune": swe.SE_NEPTUNE,
        "Pluto": swe.SE_PLUTO
    }
    
    # Set path to ephemeris files
    swe.set_ephe_path('/path/to/ephemeris/files')

    positions = {}
    
    for planet_name, planet_id in planets.items():
        position = swe.calc_ut(julian_day, planet_id, swe.FLG_SWIEPH | swe.FLG_SPEED)
        
        if position[0] < 0:
            print(f"Error calculating {planet_name}: {position[1]}")
        else:
            longitude = position[0][0]
            latitude = position[0][1]
            distance = position[0][2]
            positions[planet_name] = (longitude, latitude, distance)
    
    return positions

In [ ]:
# convert to sign/degree

In [ ]:
def convert_to_sign_and_degree(longitude):
    signs = [
        "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
        "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
    ]

    # Determine the sign and degree within the sign
    sign_index = int(longitude // 30)
    sign = signs[sign_index]
    degree = longitude % 30
    
    return sign, degree

def display_planetary_positions(positions):
    for planet_name, (longitude, latitude, distance) in positions.items():
        sign, degree = convert_to_sign_and_degree(longitude)
        print(f"{planet_name} - {sign} {degree:.2f}° | Longitude: {longitude:.6f}°, Latitude: {latitude:.6f}°, Distance: {distance:.6f} AU")

In [ ]:
# python script to calculate aspects from single date

In [ ]:
def calculate_aspects(positions):
    aspects = {
        "Conjunction": {"angle": 0, "orb": 8},
        "Opposition": {"angle": 180, "orb": 6},
        "Trine": {"angle": 120, "orb": 6},
        "Square": {"angle": 90, "orb": 6},
        "Sextile": {"angle": 60, "orb": 6},
        "Inconjunct": {"angle": 150, "orb": 4}
    }
    
    aspect_results = []
    planets = list(positions.keys())
    
    for i in range(len(planets)):
        for j in range(i + 1, len(planets)):
            planet1 = planets[i]
            planet2 = planets[j]
            
            longitude1 = positions[planet1][0]
            longitude2 = positions[planet2][0]
            speed1 = positions[planet1][3]
            speed2 = positions[planet2][3]
            
            angle_difference = abs(longitude1 - longitude2)
            if angle_difference > 180:
                angle_difference = 360 - angle_difference
            
            for aspect, data in aspects.items():
                orb = abs(angle_difference - data["angle"])
                if orb <= data["orb"]:
                    applying = "applying" if speed1 > speed2 else "fading"
                    aspect_results.append({
                        "planet1": planet1,
                        "planet2": planet2,
                        "aspect": aspect,
                        "angle": angle_difference,
                        "orb": orb,
                        "applying": applying
                    })
    
    return aspect_results

In [ ]:
# display results

In [ ]:
def display_aspects(positions, aspect_results):
    for aspect in aspect_results:
        planet1 = aspect['planet1']
        planet2 = aspect['planet2']
        sign1, degree1 = convert_to_sign_and_degree(positions[planet1][0])
        sign2, degree2 = convert_to_sign_and_degree(positions[planet2][0])
        
        print(f"{planet1}")
        print(f"\t{sign1} {degree1:.2f}°")
        print(f"{planet2}")
        print(f"\t{sign2} {degree2:.2f}°")
        print(f"{aspect['aspect']} ({planet1} -> {planet2})")
        print(f"\torb: {aspect['orb']:.2f}°")
        print(f"\t{aspect['applying'].capitalize()}")
        print()

In [ ]:
# bringing single date/location together

In [ ]:
def main():
    # Step 1: Get user input
    year, month, day, hour, minute, latitude, longitude = get_user_input()

    # Step 2: Query planetary positions
    positions = query_planetary_positions(year, month, day, hour, minute, latitude, longitude)

    # Step 3: Calculate aspects between the planets
    aspects = calculate_aspects(positions)

    # Step 4: Display the aspects and their details
    display_aspects(positions, aspects)

if __name__ == "__main__":
    main()

In [ ]:
# querry two different dates/times/locations at once

In [ ]:
def query_positions_for_two_instances(date1, date2, location1, location2):
    # Unpack the inputs for the first instance
    year1, month1, day1, hour1, minute1, latitude1, longitude1 = date1 + location1
    
    # Unpack the inputs for the second instance
    year2, month2, day2, hour2, minute2, latitude2, longitude2 = date2 + location2
    
    # Query positions for the first date/time/location
    positions1 = query_planetary_positions(year1, month1, day1, hour1, minute1, latitude1, longitude1)
    
    # Query positions for the second date/time/location
    positions2 = query_planetary_positions(year2, month2, day2, hour2, minute2, latitude2, longitude2)
    
    return positions1, positions2

In [ ]:
# calculating aspects between two different querries

In [ ]:
def calculate_aspects_between_instances_all_planets(positions1, positions2):
    aspects = {
        "Conjunction": {"angle": 0, "orb": 8},
        "Opposition": {"angle": 180, "orb": 6},
        "Trine": {"angle": 120, "orb": 6},
        "Square": {"angle": 90, "orb": 6},
        "Sextile": {"angle": 60, "orb": 6},
        "Inconjunct": {"angle": 150, "orb": 4}
    }
    
    aspect_results = []
    planets1 = list(positions1.keys())
    planets2 = list(positions2.keys())
    
    for planet1 in planets1:
        for planet2 in planets2:
            longitude1 = positions1[planet1][0]
            longitude2 = positions2[planet2][0]
            speed1 = positions1[planet1][3]
            speed2 = positions2[planet2][3]
            
            angle_difference = abs(longitude1 - longitude2)
            if angle_difference > 180:
                angle_difference = 360 - angle_difference
            
            for aspect, data in aspects.items():
                orb = abs(angle_difference - data["angle"])
                if orb <= data["orb"]:
                    applying = "applying" if speed1 > speed2 else "fading"
                    aspect_results.append({
                        "planet1": planet1,
                        "planet2": planet2,
                        "aspect": aspect,
                        "angle": angle_difference,
                        "orb": orb,
                        "applying": applying
                    })
    
    return aspect_results

In [ ]:
# display results

In [ ]:
def display_aspects_between_instances_all_planets(positions1, positions2, aspect_results):
    for aspect in aspect_results:
        planet1 = aspect['planet1']
        planet2 = aspect['planet2']
        sign1, degree1 = convert_to_sign_and_degree(positions1[planet1][0])
        sign2, degree2 = convert_to_sign_and_degree(positions2[planet2][0])
        
        print(f"{planet1}")
        print(f"\tInstance 1: {sign1} {degree1:.2f}°")
        print(f"{planet2}")
        print(f"\tInstance 2: {sign2} {degree2:.2f}°")
        print(f"{aspect['aspect']} ({planet1} -> {planet2})")
        print(f"\torb: {aspect['orb']:.2f}°")
        print(f"\t{aspect['applying'].capitalize()}")
        print()

In [ ]:
# bring two inputs together